In [ ]:
import os
import json
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
import sqlite3

load_dotenv(override=True)

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GEMINI_KEY = os.getenv("GEMINI_API_KEY")

if GROQ_API_KEY is None:
    raise ValueError("GROQ_API_KEY is not set")

if GEMINI_KEY is None:
    raise ValueError("GEMINI_KEY is not set")

gemini = OpenAI(api_key=GEMINI_KEY, base_url="https://generativelanguage.googleapis.com/v1beta")
groq = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")

CHAT_MODEL = "qwen/qwen3-32b"
VOICE_MODEL = "canopylabs/orpheus-v1-english"




In [ ]:
# initiate DB
DB = "library.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute("""CREATE TABLE IF NOT EXISTS books (
        title TEXT PRIMARY KEY,
        author TEXT,
        status TEXT,
        borrower TEXT
    )""")
    conn.commit()

In [ ]:
#define tools 

def get_book_info(title):
    print(f"Getting info about {title}...")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        SELECT * FROM books WHERE title = ?
        """, (title.lower(),))
        book = cursor.fetchone()

        if book:
            return f"Title: {book[0]} Author: {book[1]} Status: {book[2]} Borrower: {book[3]}"
        else:
            return "Book not found"

      #handle adding and updating books  
def add_book(title, author):
    print(f"Adding a book with the title {title} and author {author}...")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        INSERT INTO books (title, author, status, borrower) VALUES (?, ?, ?, ?)
        ON CONFLICT(title) DO UPDATE SET author = ?, status = ?, borrower = ?
        """, (title.lower(), author, "available", None, author, "available", None))
        conn.commit()
        return f"Book '{title}' by {author} added to the library."
    
def borrow_book(title, borrower):
    print(f"Completing a  borrow request for {borrower}. Title: {title}...")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        UPDATE books SET status = ?, borrower = ? WHERE title = ?
        """, ("borrowed", borrower, title.lower()))
        conn.commit()
        return f"Book '{title}' borrowed by {borrower}."
    
def return_book(title):
    print(f"Returning{title}...")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        UPDATE books SET status = ?, borrower = ? WHERE title = ?   
        """, ("available", None, title.lower()))
        conn.commit()
        return f"Book '{title}' returned to the library."
    
def get_all_books():
    print("Fetching all books")
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        SELECT * FROM books
        """)
        books = cursor.fetchall()
        return json.dumps([{"title": book[0], "author": book[1], "status": book[2], "borrower": book[3]} for book in books])
        

In [ ]:
#adding sample data
add_book("Python 101", "Joe Mwangi")
add_book("Clean Code", "Robert C. Martin")
add_book("Deep Learning 101", "Ian Goodfellow")
add_book("The Pragmatic Programmer", "Andrew Hunt")
add_book("Introduction to Algorithms", "Cormen, Leiserson, Rivest, Joe")

In [ ]:
get_all_books()

In [ ]:
borrow_book("Python 101", "Joe Mwangi")
get_all_books()
return_book("Python 101")
get_all_books()

In [ ]:
add_book_function = {
    "name":"add_book",
    "description": "Add or update a book record",
    "parameters": {
        "type": "object",
        "properties": {
            "book_title": {
                "type": "string",
                "descrition": "The book title",
            },
            "book_author": {
                "type": "string",
                "descrition": "The Author(s) of the book",
            },
            
        },
        "required": ["book_title", "book_author"],
        "additionalProperties": False,
    }
}

get_book_info_function = {
    "name":"get_book_info",
    "description": "Get information about a book using its title",
    "parameters": {
        "type": "object",
        "properties": {
            "book_title": {
                "type": "string",
                "description": "The title of the book the user wants information about."
            },
        },
        "required": ["book_title"],
        "additionalProperties": False,
    }
}

borrow_book_function = {
    "name":"borrow_book",
    "description": "Allow the user to borrow a book from the library",
    "parameters": {
        "type": "object",
        "properties": {
            "book_title":{ 
                "type": "string",
                "description": "The title of the book the user wants to borrow",
            },
            "book_borrower":{
                "type": "string",
                "description": "The name of the user that wants to borrow a book",
            }
        },
        "required": ["book_title", "book_borrower"],
        "additionalProperties": False,
    }
}
return_book_function = {
    "name":"return_book",
    "description": "Allows the user to return a borrowed book",
    "parameters": {
        "type": "object",
        "properties": {
            "book_title": {
                "type": "string",
                "description": "The title of the book the user wants to return"
            }
        },
        "required": ["book_title"],
        "additionalProperties": False,
    }
}

get_all_books_function = {
    "name": "get_all_books",
    "description": "Allow the user to get information about all the books.",
    "parameters": {
        "type": "object",
        "properties": {},
        "additionalProperties": False
    }
}

tools = [
    {"type": "function","function":get_all_books_function}, 
    {"type": "function","function":add_book_function}, 
    {"type": "function","function":get_book_info_function}, 
    {"type": "function","function":borrow_book_function}, 
    {"type": "function","function":return_book_function}
    ]
    
tools

In [ ]:

def put_message_in_chat(message, history):
    return "", history + [{"role": "user", "content": message}]

In [ ]:
def talker(message):
    response = groq.audio.speech.create(
        model=VOICE_MODEL,
        voice="troy",
        input=message,
        response_format="wav",
    )
    return response.content

In [ ]:
def handle_tool_calls(message):
    print("handling tool calls...")
    print(message)
    results = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "get_book_info":
            result = get_book_info(title=arguments.get("book_title"))
        elif tool_call.function.name == "add_book":
            result = add_book(title=arguments.get("book_title"), author=arguments.get("book_author"))
        elif tool_call.function.name == "borrow_book":
            result = borrow_book(title=arguments.get("book_title"), borrower=arguments.get("book_borrower"))
        elif tool_call.function.name == "return_book":
            result = return_book(title=arguments.get("book_title"))
        elif tool_call.function.name == "get_all_books":
            result = get_all_books()

        #check if tool_call_id exits 
        if tool_call.id:
            print(f"Tool call id: {tool_call.id[:5]}...")
        else:
            raise Exception("Tool call Id not found")
            
        results.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id,
        })
    return results

In [ ]:
system_prompt = """
    You are a library clerk able to allow users to check out available books and to perform actions such as getting 
    book information, adding books to the library, borrowing and returning books. You have been provided with a list
    of tools that you can call to provide the user with the requested service or information.
    Be as concise as possible giving short responses of one to two lines. 
    Be as accurate as possible. If you don't know, or cannot perfor a certain task, say so.
    Please obtain all the required arguments before calling a tool.
"""

In [ ]:
def chat(history):
    print("entering chat")
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    system = [{"role": "system", "content": system_prompt}]
    reply = gemini.chat.completions.create(
        model="gemini-3-flash-preview",
        messages=system + history,
        tools=tools,
        )
    while reply.choices[0].finish_reason == "tool_calls":
        print("calling tools")
        results = handle_tool_calls(reply.choices[0].message)
        history.extend(results)
        print(history)
        reply = gemini.chat.completions.create(
            model="gemini-3-flash-preview",
            messages=system + history,
            tools=tools,
        )
        print("adding user input to history")
    history += [{"role": "user", "content": reply.choices[0].message.content}]
    voice = talker(reply.choices[0].message.content)

    return history, voice
    

In [ ]:
# reply = gemini.chat.completions.create(
#         model="gemini-3-flash-preview",
#         messages=[{"role": "user", "content": "hello"}],
#         tools=tools,
#         )
# print(reply.choices[0].message.content)

In [ ]:
# building out the gradio ui

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with the library clerk...")

    message.submit(fn=put_message_in_chat, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        chat, inputs=[chatbot], outputs=[chatbot, audio_output]
    )

ui.launch()